In [1]:
import torch
import torch.nn as nn
from torchcrf import CRF
import pandas as pd

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
a = torch.rand(32, 8)
b = torch.zeros([32, 8], dtype=torch.float)
b[0, 0] = 1
loss = nn.BCEWithLogitsLoss(reduction='none')(a, b)

In [ ]:
# loss = torch.where(b != 0, loss, torch.tensor([0.]))
loss

In [33]:
loss = torch.where(b != 0, loss, loss*0.5)

In [37]:
loss.shape

torch.Size([32, 8])

In [44]:
num_tags = 5  # number of tags is 5
model = CRF(num_tags)

In [48]:
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags) #n_seq x batch x logits
tags = torch.tensor([
  [0, 1], [2, 4], [3, 1]
], dtype=torch.long)  #nseq x batch
loss = model(emissions, tags)



In [57]:
a = torch.rand(30, 32, 142)
b = torch.zeros(30, 32, dtype=torch.long)
model = CRF(a.shape[-1])


In [58]:
model(a, b)

tensor(-4717.1206, grad_fn=<SumBackward0>)

In [5]:
list = []
name = ['train', 'test', 'dev']
for i in name:
    for label in open("/home/xps/educate/code/hust/XQA/docs/src/JointIDSF/PhoATIS/word-level" + f'/{i}/label'):
        if label.strip() not in list:
            list.append(label.strip())

In [7]:
len(list)

28

In [9]:
train = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_train.csv", header=None)


In [14]:
a = {'vietanh': 'ngu'}
a.update({'vie anh': 'ocho'})

(array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
        'atis_ground_service', 'atis_airline', 'atis_abbreviation',
        'atis_quantity'], dtype=object),
 4834)

In [15]:
test = pd.read_csv("/home/xps/educate/code/hust/XQA/data/raw/ATIS/atis_intents_test.csv", header=None)
test[0].unique(), len(test)

(array(['atis_flight', 'atis_airfare', 'atis_ground_service',
        'atis_airline', 'atis_flight_time', 'atis_quantity',
        'atis_abbreviation', 'atis_aircraft'], dtype=object),
 800)

In [7]:
lengths = {}
with open("/home/xps/educate/code/hust/XQA/data/raw/PhoATIS/word-level/train/seq.in", 'r') as f:
    for line in f.readlines():
        if len(line.strip().split(' ')) not in lengths.keys():
            lengths[len(line.strip().split(' '))] = 1
        else:
            lengths[len(line.strip().split(' '))] += 1 
        if len(line.strip().split(' ')) == 50:
            print(line.strip()) 
    

tôi muốn một chuyến bay khứ_hồi từ đồng_hới đến côn_đảo khởi_hành vào thứ tư ngày 16 tháng 6 đến côn_đảo vào buổi tối khoảng 7 giờ và trở về đồng_hới vào buổi tối ngày hôm sau khoảng 7 giờ hãng hàng_không nào có chuyến bay phù_hợp với tuyến đường đó


In [ ]:
sorted(lengths, key = lambda d: lengths[d])

In [1]:
import os 

In [2]:
os.path.basename('/hoa/hoctro')

'hoctro'

In [60]:
import pandas as pd
os.environ['dir'] = "/home/xps/educate/code/hust/XQA/data"
from utils.preprocess import string2list, get_label
from collections import Counter
from transformers import AutoTokenizer
import torch

data_dir = os.environ['dir']
raw_dir = data_dir + '/data/raw/PhoATIS'
processed_dir = data_dir + '/data/processed/PhoATIS'
qa_processed = data_dir + '/data/processed/QA'
tokenizer = AutoTokenizer.from_pretrained('NlpHUST/bert-base-vn')

In [3]:
train_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/train.csv")

In [4]:
test_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/test.csv")

In [5]:
dev_df = pd.read_csv("/home/xps/educate/code/hust/XQA/data/processed/QA/dev.csv")

In [168]:
def get_label(off_set_map, text, start, end=False):
    '''
      if END_IDX is provided, just use it and leave TEXT=''
    '''
    start_idx, end_idx = 0, 0
    for i in range(len(off_set_map)):
        if off_set_map[i][0] == start:
            start_idx = i
        if text != '' and start_idx != 0:
            if off_set_map[i][1] == start + len(text):
              # print(i)
              end_idx = i 

    # print("what", start_idx, end_idx)
    return torch.tensor(start_idx, dtype=torch.long), torch.tensor(end_idx, dtype=torch.long)

In [171]:
for i in range(len(dev_df)):
    q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
    input = tokenizer(q.strip(), c, return_tensors='pt',
                max_length=386,
                truncation="only_second",
                return_offsets_mapping=True,
                padding="max_length",)
    a, t = string2list(a, 'int'), string2list(t)
    for i in range(len(a)):
        start, end = get_label(input['offset_mapping'][0], t[i], a[i])
        if start == 0 or end == 0:
            print(a[i], t[i])
            break




449 Quận 12 ở phía Đông
194 vua Clovi
485 đứng thứ 17 trên tổng số 20 đô thị lớn của thế giớ
469 dân số quá đông, tâm lý stress của đô thị, ô nhiễm, giá cả đắt đỏ, kém an ninh
144 Richelieu " là thư viện cổ, nhỏ hơn, nằm ở trung tâm, còn " François-Mitterrand " là một công trình kiến trúc hiện đại gồm bốn tòa nhà cao tầng ở khu Paris Rive Gauche
577 Những sách, tạp chí, truyện tranh... được mượn tự do, còn các đĩa nhạc, video thì cần trả một số tiền trung bình hàng năm


KeyboardInterrupt: 

In [155]:
for i in input['offset_mapping'][0]:
    if i[0] == a[0]:
        print(i)

tensor([6, 9])


In [159]:
start, end = get_label(input['offset_mapping'][0], t[0], a[0])
start, end

29
what 14 29


(tensor(14), tensor(29))

In [163]:
input.input_ids

tensor([[   2, 2465,  375,  143,   91,    8, 2326,   10,  521,  347,  286,  417,
            3, 2326,  790,   34,  132,  511,  469,    8,    9,   82,  100,  231,
          527,  153,  106,    7,  153,  811,    5,    7,   11,  258,  176,    8,
           20,  399,  415,   87, 1383,   10,    6, 1769,  140,  642,  432,    5,
         2326,   16,   11,   20,   12,   25,   55,  255,   69,    8,  867,  144,
            9,  550,  249, 1603,   65,    5,    9, 1966,  443,    7,   62, 1791,
            6,  704,  140,  642,  783,    5, 2326,  317,   55,   20,   12,   25,
          258,  176,    8,  677,  994,   27,  350,  489,    7,  313,  461,    6,
          912,  642, 1176,    5, 2326,   11,   55,  255,   45,  208, 1429,   92,
            8,  969, 5118,  350,    7,   12,    9,  140,  642, 1131,    5, 1287,
            5,  204,   11,  411,  265,   39, 1566,  402,  867,  144,   53,   61,
          370,  287,  203,   45,  208,    8,  867,    7,  677,  994,    6,  940,
          140,  642, 1353,  

In [165]:
tokenizer.decode(input.input_ids[0][start.item():end.item()+1])

'nằm ở điểm gặp nhau của các hành trình thương mại đường bộ và đường sông'

In [125]:
length = []

for i in range(len(dev_df)):
    q, a, c, t = dev_df.iloc[i][['question', 'start', 'context', 'text']]
    a, c = string2list(a, 'int'), string2list(c)
    length.append(len(q.strip().split()))

In [108]:
sum(length)/len(length)

13.979868708971553

In [39]:
a, c

([432, 432, 432, 432],
 ['Năm 1871, Đức trở thành một quốc gia dân tộc khi hầu hết các quốc gia Đức thống nhất trong Đế quốc Đức do Phổ chi phối. Sau Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919, Đế quốc này bị thay thế bằng Cộng hòa Weimar theo chế độ nghị viện. Chế độ độc tài quốc xã được hình thành vào năm 1933, dẫn tới Chiến tranh thế giới thứ hai và một nạn diệt chủng. Sau một giai đoạn Đồng Minh chiếm đóng, hai nước Đức được thành lập: Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức. Năm 1990, quốc gia được tái thống nhất.'])

In [4]:
max = 0
for i in range(len(df)):
    item = df.iloc[i]
    start = string2list(item['start'], type='int')
    text = string2list(item['text'])
    break

In [24]:
a = "wefwefwefwfe@@@dfsdfsdf"
a.split("@@@")

['wefwefwefwfe', 'dfsdfsdf']

In [ ]:
from transformers import RobertaModel, AutoTokenizer
model_checkpoint = 'vinai/phobert-base'
# model = QAModule(model_checkpoint=model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
input = tokenizer("DIT CON ME", return_tensors='pt',
        max_length=258,
        truncation="only_second",
        padding="max_length",)
model(input_ids=input.input_ids.repeat(16, 1), attention_mask=input.attention_mask.repeat(16, 1))


In [1]:
model.device

NameError: name 'model' is not defined